# Artificial Intelligence
## Semester Project Phase I
### Muhammad Zain Ali--------22i-0562 
### Syed Asad Shah-------------22i-0597 

In [1]:
# Necessary Libraries
import numpy as np
import pandas as pd
import random

In [2]:
# Reading all the CSV Files and converting them into Pandas Data Frames
Courses = pd.read_csv('courses.csv')
St_Course = pd.read_csv('studentCourse.csv')
Students = pd.read_csv('studentNames.csv')
Teachers = pd.read_csv('teachers.csv')

### Hard Constraints
• An exam will be scheduled for each course.

• A student is enrolled in at least 3 courses. A student cannot give more than 1 exam at a
time.

• Exam will not be held on weekends.

• Each exam must be held between 9 am and 5 pm.

• Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams
at the same time.

• A teacher cannot invigilate two exams in a row.
### Soft Constraints
• All students and teachers shall be given a break on Friday from 1-2.

• A student shall not give more than 1 exam consecutively.

• If a student is enrolled in a MG course and a CS course, it is preferred that their
MG course exam be held before their CS course exam.

• Two hours of break in the week such that at least half the faculty is free in one slot
and the rest of the faculty is free in the other slot so the faculty meetings shall be
held in parts as they are now.

In [3]:
#Creating a list of Courses 
Courses_List = []
for i in range(Courses.shape[0]):
    Courses_List.append(Courses.iloc[i, 0])

Teachers_List = []
for i in range(Teachers.shape[0]):
    Teachers_List.append(Teachers.iloc[i, 0])

# Time Table Dataframe:
TimeTable = {
    'Time': ['09:00 AM C301', '10:00 AM C302', '11:00 AM C401', '12:00 PM C402', '01:00 PM C303', '02:00 PM C408', '03:00 PM C404', '04:00 PM C409'],
    'Monday' : '',
    'Tuesday' : '',
    'Wednesday' : '',
    'Thursday' : '',
    'Friday': ''
}

TimeTable = pd.DataFrame(TimeTable) A Function to Check the Probablility of the Gaps and Breaks
def Check_Prob():
    return (round(random.random(), 2))

# Setting the specific Gap Counts to the Days
gap_count = {'Monday': 2, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 3, 'Friday': 2}
Courses_List_Copy = Courses_List.copy()
Teachers_List_Copy = Teachers_List.copy()
# Inserting the Values in the Data Frame of The Timetable
for i in range(5):
    # A Bool Variable to check if the previous entry was also a gap or not
    prev_entry_was_gap = False
    for j in range(8):
        #Constraint for Jumma Break
        if (i+1 == 5 and j == 4):
            TimeTable.iloc[j, i+1] = 'BREAK'
        else:
            # Selecting a random Course from the Course list
            if len(Courses_List_Copy) != 0:
                Selected_Course = random.choice(Courses_List_Copy)
                Selected_Teacher = random.choice(Teachers_List_Copy)
            else:
                # if sll the Courses are inserted in the Data Frame, It will Break
                break

            day = TimeTable.columns[i+1]
            # Setting the probabilty rate 0.4 and entering the course
            if Check_Prob() > 0.4 and gap_count[day] > 0 and not prev_entry_was_gap:
                TimeTable.iloc[j, i+1] = ''
                gap_count[day] -= 1
                prev_entry_was_gap = True
            else:
                # Removing the Entered course from the Course list
                TimeTable.iloc[j,i+1] = Selected_Course + ' ' + Selected_Teacher
                Courses_List_Copy.remove(Selected_Course)
                prev_entry_was_gap = False

In [4]:
# Displaying the Time table 
display(TimeTable.style.hide_index())

/tmp/ipykernel_3057/3236398665.py:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  display(TimeTable.style.hide_index())


Time,Monday,Tuesday,Wednesday,Thursday,Friday
09:00 AM C301,CS118 Gul e Aisha,,,,
10:00 AM C302,,SS152 Mehboobullah,DS3011 Zainab Moin,CS219 Waseem Shahzad,CS328 Shahzad Mehmood
11:00 AM C401,EE229 Maheen Arshad,CS211 Shafaq Riaz,CS217 Mehboobullah,AI202 Nagina Safdar,MT205 Sara Aziz
12:00 PM C402,SS111 Kifayat Ullah,,CS328 Mehboobullah,SE110 Noor ul Ain,MT224 Sanaa Ilyas
01:00 PM C303,,CS302 Umair Arshad,,,BREAK
02:00 PM C408,CS220 Asif Naeem,CY2012 Maimoona Rassol,EE227 Zainab Abaid,CS211 Asma Nisa,
03:00 PM C404,SS113 Hasan Mujtaba,CS218 Javaria Imtiaz,,SS118 Arshad Islam,AI2011 Usman Ashraf
04:00 PM C409,CS307 Sara Aziz,MT210 Shehreyar Rashid,MG220 Sanaa Ilyas,,MG223 Asma Nisa


In [82]:
student_course_counts = St_Course.groupby('Student Name')['Course Code'].nunique()

# Checking the students who enrolled in less than 3 courses
students_with_less_than_3_courses = student_course_counts[student_course_counts < 3].index.tolist()

# Enrolling them in additional courses until they are enrolled in at least 3 courses
new_rows = []
for student in students_with_less_than_3_courses:
    courses_needed = 3 - student_course_counts[student]
    additional_courses = St_Course[~St_Course['Course Code'].isin(St_Course[St_Course['Student Name'] == student]['Course Code'].tolist())]['Course Code'].sample(n=courses_needed, replace=True)
    for course in additional_courses:
        new_rows.append({'Student Name': student, 'Course Code': course})

# Updating the DataFrame with the additional enrollments
St_Course = pd.concat([St_Course, pd.DataFrame(new_rows)], ignore_index=True)

print(St_Course.groupby('Student Name').size())
print("\n\nAll students are enrolled in at least 3 courses.")
St_Course

Student Name
Abdul Gafur                3
Adam C Ryan                3
Adam Flowers               3
Adam Green                 3
Adam Kirk                  3
                          ..
Zahir Hussain              3
Zahra Faraji Rad           3
Zainab Khattak             3
Zaki Choudhury             3
Zeenat K Guness-Goolbar    4
Length: 168, dtype: int64


All students are enrolled in at least 3 courses.


,Student Name,Course Code
0,Sam D Edwards,AI2011
1,Sheila Hughton,DS3011
2,Yasmin Ahmed,SE110
3,Sarah N Md Sallehuddin Khan,EE229
4,Sarah Nolasco,AI2011
...,...,...
507,Yasir M Salih,SS118
508,Zahir Hussain,SE110
509,Zainab Khattak,CS219
510,Zaki Choudhury,AI2011


# Genetic Algorithm

### Initial Population

In [83]:
# Initial Population for the GA is the Current Time Table
Initial_Population = []
Initial_Population.append(TimeTable)

### Fitness Function

In [84]:
# An exam will be scheduled for each course.
def Check_Courses_in_TimeTable(Dataframe):
    for i in range(5):
        for j in range(8):
            Course = Dataframe.iloc[j, i+1]
            for item in Courses_List:
                if Course in item:
                    Courses_List.remove(item)

    if len(Courses_List) == 0:
        return 1
    else:
        return 0

# A student is enrolled in at least 3 courses.
def Check_Students_Enrollment():
    if True:
        return 1
    else:
        return 0
    
# A student cannot give more than 1 exam at a time.
def Exam_atmost1():
    # Creating a set to keep track of students taking exams at each time slot
    students_at_time = set()

    constraint = False

    # Iterating over each row (time slot) in the timetable
    for index, row in TimeTable.iterrows():
        time = row['Time']
        for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            student = row[day]
            if student != '':
                # Checking if the student is already taking an exam at this time
                if (time, student) in students_at_time:
                    constraint_violated = True
                else:
                    # Adding the student to the set of students taking exams at this time
                    students_at_time.add((time, student))

    if not constraint:
        return 1
    else:
        return 0
    
# Exam will not be held on weekends.
def ExamDays(Dataframe):
    columns_list = list(Dataframe.columns)
    if 'Saturday' not in columns_list and 'Sunday' not in columns_list:
        return 1
    else:
        return 0
    
# Each exam must be invigilated by a teacher.
def Teacher_Assigned():
    return 1

# A teacher cannot invigilate two exams at the same time.
def Teachers_atmost1(Dataframe):
    # Creating a set to keep track of teachers invigilating exams at each time slot
    teachers_at_time = set()

    constraint = False

    # Iterating over each row (time slot) in the timetable
    for index, row in TimeTable.iterrows():
        time = row['Time']
        for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            teacher = row[day]
            if teacher != '':
                # Checking if the teacher is already invigilating an exam at this time
                if (time, teacher) in teachers_at_time:
                    constraint = True
                else:
                    # Adding the teacher to the set of teachers invigilating exams at this time
                    teachers_at_time.add((time, teacher))

    if not constraint:
        return 1
    else:
        return 0

def Teachers_in_Row(Dataframe):
    # Creating a variable to keep track of the teacher invigilating the previous exam
    previous_teacher = None

    constraint= False

    # Iterating over each row (time slot) in the timetable
    for index, row in TimeTable.iterrows():
        time = row['Time']
        for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            teacher = row[day]
            if teacher != '':
                # Checking if the teacher is the same as the previous time slot
                if teacher == previous_teacher:
                    constraint = True
                else:
                    # Updating the previous_teacher variable
                    previous_teacher = teacher

    if not constraint:
        return 1
    else:
        return 0

def Friday_Break(Dataframe):
    if Dataframe.iloc[4,5].lower() == 'break':
        return 1
    else:
        return 0
    
def MG_BEFORE_CS():
    

# Fitness Function for the Genetic Algorithm    
def Fitness(Dataframe):
    Fitness_Value = 0
    
    Fitness_Value = Fitness_Value + Check_Courses_in_TimeTable(Dataframe) + Check_Students_Enrollment() + Exam_atmost1() + ExamDays(Dataframe)
    Fitness_Value = Fitness_Value + Teachers_in_Row(Dataframe) + Teacher_Assigned() + Teachers_atmost1(Dataframe) + Friday_Break(Dataframe)
    if Fitness_Value != None:
        return Fitness_Value
    

### Crossover

In [85]:
def Crossover(Dataframe1, Dataframe2):
    # Check if inputs are DataFrame objects
    if not isinstance(Dataframe1, pd.DataFrame) or not isinstance(Dataframe2, pd.DataFrame):
        raise ValueError("Inputs must be DataFrame objects")

    New_Dataframe = pd.concat([Dataframe1[['Time', 'Monday', 'Tuesday']], Dataframe2[['Wednesday', 'Thursday', 'Friday']]], axis=1)
    return New_Dataframe


### Mutation

In [86]:
def Mutation(df):
    count = 0
    for i in range(5):
        for j in range(8):
            if count <= 6:
                if j != 7:
                    if Check_Prob() > 0.4:
                        temp = df.iloc[j, i+1]
                        df.iloc[j, i+1] = df.iloc[j+1, i+1]
                        df.iloc[j+1, i+1] = temp
                        count += 1
    return df

### GA Loop

In [87]:
Total_gen = 10
Selected_Parents = []
for gen in range(Total_gen):
    # Evaluating the Fitness for the TimeTable
    fitness_value = Fitness(TimeTable)
    
    # Selecting parents for the next generation
    if gen == 0:
        Dataframe1 = TimeTable.copy()
        Initial_Population.append(Dataframe1)
        Dataframe2 = TimeTable.copy()
        Initial_Population.append(Dataframe2)

    # Roulette Wheel Selection
    New_Individuals = random.sample(Initial_Population, 1)
    Selected_Parents.append(max(New_Individuals, key=Fitness))
    
    # Applying crossover to create offspring
    Dataframe1 = random.sample(Selected_Parents, 1)
    Dataframe2 = random.sample(Selected_Parents, 1)
    New_Offspring = Crossover(Dataframe1[0], Dataframe2[0])

    # Applying mutation to introduce variation in the offspring
    Mutated_Offspring = Mutation(New_Offspring)
    Population.append(Mutated_Offspring)
    
# The Fittest Offspring for the Current Output:
display(Initial_Population[-1].style.hide_index())
print(f"Fitness Value {Fitness(Initial_Population[-1])}.")

/tmp/ipykernel_38052/153805800.py:27: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  display(Initial_Population[-1].style.hide_index())


Time,Monday,Tuesday,Wednesday,Thursday,Friday
09:00 AM C301,MT210 Gul e Aisha,,,,
10:00 AM C302,SS113 Rohail Gulbaz,CS328 Umair Arshad,SS118 Subhan Ullah,CS328 Arshad Islam,MT224 Khadija Farooq
11:00 AM C401,AI2011 Javaria Imtiaz,,,DS3011 Amna Irum,MT205 Farah Jabeen Awan
12:00 PM C402,,CS211 Farwa Batool,CS307 Irum Inayat,EE229 Ameen Chilwan,
01:00 PM C303,MG220 Hassan Raza,CS211 Adnan Tariq,CS302 Farwa Batool,,BREAK
02:00 PM C408,,MG223 Hassan Raza,,SE110 Muhammad bin Qasim,CS217 Shafaq Riaz
03:00 PM C404,CY2012 Sara Aziz,CS220 Muhammad bin Qasim,CS218 Shafaq Riaz,,EE227 Sajid Khan
04:00 PM C409,SS111 Shehreyar Rashid,SS152 Sara Aziz,AI202 Gul e Aisha,CS118 Umair Arshad,CS219 Shafaq Riaz


Fitness Value 8.
